In [1]:
import pandas as pd
import datetime
from gsheets_connect import GoogleSheets, Connect
import helper_functions as hlp
from expense import Expense

In [2]:
from credit_card_object import CreditCard

In [3]:
mnths_dict = {
    'Ene':'01','Feb':'02','Mar':'03',
    'Abr':'04','May':'05','Jun':'06',
    'Jul':'07','Ago':'08','Sep':'09',
    'Oct':'10','Nov':'11','Dic':'12'
}

In [20]:
data = pd.read_html('5470467005458188 - 17_Ene_20.xls')

cols = data[1].loc[5,:].drop(1)
cols.name = 'ix'

data = data[1].loc[6:,:].drop(1, axis=1)

data.columns = cols
data.reset_index(inplace=True, drop=True)

In [5]:
from datetime import datetime

In [6]:
d = data.loc[0, 'FECHA']

In [7]:
def parse_month(date_string):
    for m,v in mnths_dict.items():
        if m in date_string:
            date_string = date_string.replace(m,v)
    return date_string

In [8]:
data.iloc[:,0] = data.iloc[:,0].apply(
    lambda d: parse_month(d)
).apply(
    lambda d: datetime.strptime(d, '%d/%m/%Y')
)

data.iloc[:,2] = data.iloc[:,2].astype(float)

In [9]:
positive_data = data.loc[data.iloc[:,2]>0]

In [10]:
cols = [
    "Amount $",
    "Expense Day", 
    "Expense Month", 
    "Payment Day", 
    "Payment Month", 
    "Installments", 
    "Payment Method",
    "Method Name",
    "Category", 
    "Sub Category",
    "Payment Fortnight",
    "Description"
]
new_expenses_df = pd.DataFrame(index=[1,2,3],columns=cols)

In [11]:
new_expenses_df.index.max()

3

In [12]:
GSHEET_ID = '14a36tesQZ2AH0aIEdG5Ch2F8iVyrnXCvlxeASLX47N4'

In [15]:
def update_data_frame(cur_df, expenses):
    
    for expense in expenses:
        lst_ix = 0 if cur_df.shape[0] == 0 else cur_df.index.max()+1
        new_df = pd.DataFrame(index=[lst_ix], columns=cur_df.columns)

        new_df.iloc[0].loc["Expense Day"] = expense.getDateAsString()
        new_df.iloc[0].loc["Expense Month"]=expense.getMonthNum()
        new_df.iloc[0].loc["Payment Day"]=expense.getPaymentDateAsString()
        new_df.iloc[0].loc["Payment Month"]=expense.getPaymentMonthNum()
        new_df.iloc[0].loc["Installments"]=expense.getInstallments()
        new_df.iloc[0].loc["Payment Method"]=expense.getPaymentMethod()
        new_df.iloc[0].loc["Method Name"]=expense.getPaymentMethodName()
        new_df.iloc[0].loc["Category"]=expense.getMainCategory()
        new_df.iloc[0].loc["Sub Category"]=expense.getSubCategory()
        new_df.iloc[0].loc["Amount $"]=expense.getAmount()
        new_df.iloc[0].loc["Payment Fortnight"]=expense.getPaymentFortnight()
        new_df.iloc[0].loc["Description"]=expense.getDescription()
        
        cur_df = pd.concat([cur_df, new_df])
    
    return cur_df

def modifyInfo(an_expense, row, main_cats, sub_cats):
    print("""
    Registering a new expense. At any time:
        * r is for restart, and it restarts 
            the entire expense information logging, 
        * q kills the entire process.
    """)
    
    try:
        date, concept, amount = row

        cat, scat = hlp.selectCategory(main_cats, sub_cats)
        installments = hlp.askForInstallments(
            is_credit = type(pm) == CreditCard
        )

        an_expense.updateData(
            payment_date=date, 
            description=concept, 
            category=cat, 
            sub_category=scat, 
            amount=amount, 
            payment_method='credit' if type(pm) == CreditCard else pm, 
            n_installments=installments,
            credit_card_used=pm if type(pm) == CreditCard else None
        )

        print(an_expense.toString())
        
        sub_expenses = new_expense.divideExpense()
        print([se.getAmount() for se in sub_expenses])
        
        return True
    except RuntimeError:
        print('Quit')
        return False
    except Exception:
        return modifyInfo(an_expense, row, main_cats, sub_cats)

In [19]:
new_expenses = []
gsheet = GoogleSheets()
main_cats, sub_cats = hlp.getCategoriesFromGSheet(gsheet, GSHEET_ID)

pm = hlp.askForPaymentMethod()
new_expenses_df = pd.DataFrame(columns=cols)
positive_data = data[data['IMPORTE']>0].reset_index(drop=True).copy()
ix=0
while ix <= positive_data.shape[0]:
    if ix == -1: break
    row = data.loc[ix,:]
    print('')
    print('----------------------------------')
    print("Currently Processing this expense:")
    print(row.to_string())
    yes = hlp.askYesOrNo("Log this expense? ")
    
    if yes:       
        an_expense = Expense()
        if modifyInfo(an_expense, row, main_cats, sub_cats):
            new_expenses_df = update_data_frame(new_expenses_df, an_expense)
        else:
            break
    ix+=1


Which payment method was used? 
0) Santander-Free
1) Bancomer-Platinum
2) debit
3) cash

Enter the number of the method: 0
Santander-Free


----------------------------------
Currently Processing this expense:
ix
FECHA          2020-01-15 00:00:00
CONCEPTO    S WORLD PATRIOTISMO VD
IMPORTE                      19430
Log this expense? n

----------------------------------
Currently Processing this expense:
ix
FECHA          2020-01-15 00:00:00
CONCEPTO    S WORLD PATRIOTISMO VD
IMPORTE                       2459
Log this expense? n

----------------------------------
Currently Processing this expense:
ix
FECHA       2020-01-14 00:00:00
CONCEPTO        SUPERAMA DAKOTA
IMPORTE                   167.2
Log this expense? y

    Registering a new expense. At any time:
        * r is for restart, and it restarts 
            the entire expense information logging, 
        * q kills the entire process.
    
Select one Category from the below list:
1.  Food
2.  Getting Around
3.  Fun Stuff
4. 

Enter number of the Category: 5
Select one Sub Category from the below list:
1.  Clothing
2.  Gifts
3.  Haircut
4.  Stuff for me
5.  Personal Care
6.  Other

Enter number of the Sub Category: 4

How many installments? 1
Expense:
    Date: 2020-01-03
    Category: Personal Stuff
    Sub Category: Stuff for me
    Payment Method: credit
    Installments: 1
    Amount: $360.0

----------------------------------
Currently Processing this expense:
ix
FECHA                  2020-01-03 00:00:00
CONCEPTO    IMPTO.COMISION POR PAGO TARDIO
IMPORTE                               57.6
Log this expense? y

    Registering a new expense. At any time:
        * r is for restart, and it restarts 
            the entire expense information logging, 
        * q kills the entire process.
    
Select one Category from the below list:
1.  Food
2.  Getting Around
3.  Fun Stuff
4.  Health Care
5.  Personal Stuff
6.  Apartment Spends
7.  Removed From Savings

Enter number of the Category: 5
Select one Sub Cat

Enter number of the Category: 2
Select one Sub Category from the below list:
1.  Gas
2.  Parking
3.  Other

Enter number of the Sub Category: 3

How many installments? 1
Expense:
    Date: 2019-12-27
    Category: Getting Around
    Sub Category: Other
    Payment Method: credit
    Installments: 1
    Amount: $134.24

----------------------------------
Currently Processing this expense:
ix
FECHA          2019-12-26 00:00:00
CONCEPTO    STARBUCKS APT VALLARTA
IMPORTE                        196
Log this expense? y

    Registering a new expense. At any time:
        * r is for restart, and it restarts 
            the entire expense information logging, 
        * q kills the entire process.
    
Select one Category from the below list:
1.  Food
2.  Getting Around
3.  Fun Stuff
4.  Health Care
5.  Personal Stuff
6.  Apartment Spends
7.  Removed From Savings

Enter number of the Category: 1
Select one Sub Category from the below list:
1.  Super Market
2.  Restaurants
3.  Take-out and bin

In [22]:
new_expenses_df

,Amount $,Expense Day,Expense Month,Payment Day,Payment Month,Installments,Payment Method,Method Name,Category,Sub Category,Payment Fortnight,Description
0,167.2,2020-01-14,202001,2020-02-15,202002,1,credit,Santander-Free,Food,Super Market,2020-02-15,SUPERAMA DAKOTA
1,2300,2020-01-13,202001,2020-02-15,202002,1,credit,Santander-Free,Personal Stuff,Stuff for me,2020-02-15,CLIP MX*SERVICIOS PROF
2,515.24,2020-01-13,202001,2020-02-15,202002,1,credit,Santander-Free,Food,Super Market,2020-02-15,SUPERAMA DAKOTA
3,633,2020-01-13,202001,2020-02-15,202002,1,credit,Santander-Free,Fun Stuff,Hobbies,2020-02-15,ZAP ADICTT MERRELL PAT
4,785,2020-01-10,202001,2020-01-15,202001,1,credit,Santander-Free,Personal Stuff,Stuff for me,2020-01-15,SKIN CASE
5,608,2020-01-10,202001,2020-01-15,202001,1,credit,Santander-Free,Personal Stuff,Stuff for me,2020-01-15,SKIN CASE
6,129.8,2020-01-10,202001,2020-01-15,202001,1,credit,Santander-Free,Health Care,Supplements,2020-01-15,GNC WTC B JUAREZ
7,586.48,2020-01-10,202001,2020-01-15,202001,1,credit,Santander-Free,Food,Super Market,2020-01-15,SUPERAMA DAKOTA
8,360,2020-01-03,202001,2020-01-15,202001,1,credit,Santander-Free,Personal Stuff,Stuff for me,2020-01-15,COMISION POR PAGO TARDIO
9,57.6,2020-01-03,202001,2020-01-15,202001,1,credit,Santander-Free,Personal Stuff,Stuff for me,2020-01-15,IMPTO.COMISION POR PAGO TARDIO


In [24]:
values_list = hlp.dataFrameToListOfValues(new_expenses_df)
gsheet.values_to_gsheet(
    spreadsheet_id = GSHEET_ID,
    values_list=values_list, 
    range_name='data_dev'
)

264 cells updated.


In [21]:
data

ix,FECHA,CONCEPTO,IMPORTE
0,15/Ene/2020,S WORLD PATRIOTISMO VD,19430.00
1,15/Ene/2020,S WORLD PATRIOTISMO VD,2459.00
2,15/Ene/2020,PAGOS INTERBANCARIOS,-9026.76
3,14/Ene/2020,SUPERAMA DAKOTA,167.20
4,13/Ene/2020,CLIP MX*SERVICIOS PROF,2300.00
5,13/Ene/2020,SUPERAMA DAKOTA,515.24
6,13/Ene/2020,ENERGY FITNESS WTC,1058.25
7,13/Ene/2020,ZAP ADICTT MERRELL PAT,633.00
8,10/Ene/2020,SKIN CASE,785.00
9,10/Ene/2020,SKIN CASE,608.00


In [ ]:
try:
    raise_error(2)
except RuntimeError as e:
    print(type(e))
    print(e.)
    print(str(e) == 'ESCROM1')
    if e == 'ESCROM1':
        print('catched')

In [ ]:
GSHEET_ID = '14a36tesQZ2AH0aIEdG5Ch2F8iVyrnXCvlxeASLX47N4'

In [ ]:
gsheet = GoogleSheets()

In [ ]:
data = hlp.retrieveDataFromSheet(
    gsheet, 
    GSHEET_ID, 
    sheet_name = 'automatic spends loader'
)

In [ ]:
import arrow

In [ ]:
data['escrom'] = data['Date'].apply(lambda x: arrow.get)

In [ ]:
cur_data = pd.read_csv(r'daily_data.txt', sep=' ')

#cur_data['Payment Day'] = pd.to_datetime(cur_data['Payment Day']).dt.date

cur_data['Payment Fortnight'] = cur_data['Payment Day'].apply(
    lambda date: str(pd.to_datetime(date).replace(day=15 if pd.to_datetime(date).day <= 15 else 30).date())
)

In [ ]:
hlp.dataFrameToListOfValues(cur_data)

In [ ]:
gsheet.values_to_gsheet(
    gsheet_id = GSHEET_ID,
    values_list = hlp.dataFrameToListOfValues(cur_data).tolist(),  
    range_start='data'
)

In [ ]:
cols = np.array(cur_data.columns)

cols_values = cols.tolist()

values_arr = cur_data.values

list_of_rows = np.insert(values_arr, 0, cols_values, 0)

In [ ]:
request = gsheet.service.spreadsheets().values().clear(
    spreadsheetId=GSHEET_ID, 
    range='bimonthly data' 
    #body=clear_values_request_body
)
response = request.execute()

In [ ]:
categories_titles = gsheet.gsheet_to_df(
    spreadsheet_id = '14a36tesQZ2AH0aIEdG5Ch2F8iVyrnXCvlxeASLX47N4',
    range_name = 'biweekly plan!B27:D100'
)

In [ ]:
cur_data = gsheet.gsheet_to_df(
    spreadsheet_id = '14a36tesQZ2AH0aIEdG5Ch2F8iVyrnXCvlxeASLX47N4',
    range_name = 'data'
)

In [ ]:
cur_data.columns

In [ ]:
separations = categories_titles[
    categories_titles['Variable Expenses'].isnull()
].index

In [ ]:
separations#[:-1]

In [ ]:
cats = {}
sub_cats = {}
for i in range(1,len(separations)):
    ix_1 = separations[i-1]+1
    ix_2 = separations[i]-1
    cat = categories_titles.loc[ix_1:ix_2, 'Variable Expenses']
    cats[i] =  cat.iloc[0]
    sc = {}
    for j, c in enumerate(cat.iloc[1:]):
        sc[j+1] = c
    sub_cats[cat.iloc[0]] = sc
cat = categories_titles.loc[separations[-1]+1:, 'Variable Expenses']
cats[len(separations)] =  cat.iloc[0]
sc = {}
for j, c in enumerate(cat.iloc[1:]):
    sc[j+1] = c
sub_cats[cat.iloc[0]] = sc

In [ ]:
cats_template = hlp.categoriesTemplate(sub_cats).sort_index()
cats_template['range'] = np.arange(1, cats_template.shape[0]+1)

In [ ]:
pivot = pd.pivot_table(
    cur_data, 
    values = 'Amount $',
    columns='Payment Fortnight',
    aggfunc='sum',
    index=['Category', 'Sub Category']
).reset_index()

In [ ]:
data_with_all_cats = pd.merge(
    cats_template, 
    pivot, 
    on = ['Category', 'Sub Category'], 
    how='left'
).fillna(0)

pivoted_to_write_on_gsheet = pd.DataFrame(
    data_with_all_cats.set_index(
        ['Category', 'Sub Category', 'range']
    ).stack()
).reset_index().rename(
    columns={
        'level_3': 'Payment Fortnight', 
        0: 'Amount $'
    }
)

In [ ]:
pivoted_to_write_on_gsheet = pivoted_to_write_on_gsheet.loc[
    :,
    ['range', 'Category', 'Sub Category', 'Payment Fortnight', 'Amount $']
]

In [ ]:
cols = np.array(pivoted_to_write_on_gsheet.columns)

cols_values = cols.tolist()

values_arr = pivoted_to_write_on_gsheet.values#.tolist()

list_of_rows = np.insert(values_arr, 0, cols_values, 0)

In [ ]:
data = [
    {
        'range': 'biweekly data!B2',
        'values': list_of_rows.tolist()
    }
]
body = {
    'valueInputOption': 'RAW',
    'data': data
}
result = service.spreadsheets().values().batchUpdate(
    spreadsheetId='14a36tesQZ2AH0aIEdG5Ch2F8iVyrnXCvlxeASLX47N4', body=body).execute()
print('{0} cells updated.'.format(result.get('totalUpdatedCells')))

In [ ]:
cur_data.columns

In [ ]:
monthly_pivot = pd.pivot_table(
    cur_data, 
    values = 'Amount $',
    columns='Payment Month',
    aggfunc='sum',
    index=['Category', 'Sub Category']
).reset_index()

In [ ]:
data_with_all_cats = pd.merge(
    cats_template, 
    monthly_pivot, 
    on = ['Category', 'Sub Category'], 
    how='left'
).fillna(0)

pivoted_to_write_on_gsheet = pd.DataFrame(
    data_with_all_cats.set_index(
        ['Category', 'Sub Category', 'range']
    ).stack()
).reset_index().rename(
    columns={
        'level_3': 'Payment Month', 
        0: 'Amount $'
    }
)

In [ ]:
pivoted_to_write_on_gsheet

In [ ]:
pivoted_to_write_on_gsheet = pivoted_to_write_on_gsheet.loc[
    :,
    ['range', 'Category', 'Sub Category', 'Payment Month', 'Amount $']
]

In [ ]:
cols = np.array(pivoted_to_write_on_gsheet.columns)

cols_values = cols.tolist()

values_arr = pivoted_to_write_on_gsheet.values#.tolist()

list_of_rows = np.insert(values_arr, 0, cols_values, 0)

In [ ]:
data = [
    {
        'range': 'monthly data!B2',
        'values': list_of_rows.tolist()
    }
]
body = {
    'valueInputOption': 'RAW',
    'data': data
}
result = service.spreadsheets().values().batchUpdate(
    spreadsheetId='14a36tesQZ2AH0aIEdG5Ch2F8iVyrnXCvlxeASLX47N4', body=body).execute()
print('{0} cells updated.'.format(result.get('totalUpdatedCells')))

In [ ]:
monthly_data#.unstack(0)

In [ ]:
monthly_data.unstack(0).shape

In [ ]:
numberToLetters(12)

In [ ]:
rows_range = 

In [ ]:
dates = np.array(monthly_data.unstack(0).columns.levels[1])

cols_values = np.array(['category', 'sub_category']+dates.astype(str).tolist())

values_arr = monthly_data.unstack(0).fillna(0).reset_index().values#.tolist()

list_of_rows = np.insert(values_arr, 0, cols_values, 0)

In [ ]:
list_of_rows.shape

In [ ]:
start_from = 'biweekly pivot!B2'

In [ ]:
table_range

In [ ]:
data = [
    {
        'range': start_from,
        'values': list_of_rows.tolist()
    }
]
body = {
    'valueInputOption': 'RAW',
    'data': data
}
result = service.spreadsheets().values().batchUpdate(
    spreadsheetId='14a36tesQZ2AH0aIEdG5Ch2F8iVyrnXCvlxeASLX47N4', body=body).execute()
print('{0} cells updated.'.format(result.get('totalUpdatedCells')))

In [ ]:
list_of_rows